In [58]:
import diagonal_b6 as b6
import pandas as pd
from tqdm.notebook import tqdm

In [59]:
w = b6.connect_insecure("0.0.0.0:8002")

We use _#amenity=school_ as it has details like "name".

In [60]:
schools = b6.find(b6.tagged("#amenity", "school"))
school_names = w(schools.map(lambda f: f.get_string("name")))

In [61]:
# w(schools)[0][1].all_tags()

In [62]:
# Pick out EMAS, as we want to center our analysis around that.
[emas_feature_id] = [ s for s, n in school_names if "Montessori" in n ]

In [63]:
emas_feature_id

/point/openstreetmap.org/node/4057361549

In [34]:
results = []
new_tags = []
our_crossings = []

# Note: Should we look for more crossings?
# For example, adding on:
#
#   > .or_(b6.keyed("#highway", "crossing"))
# ?
is_crossing = b6.keyed("crossing") # 

for school, name in tqdm(school_names):

    # New feature: 'is_valid()'
    crossings = w( b6.accessible_all([school], is_crossing, 500.0, {"mode": "walk"})
                  .filter(lambda x: b6.matches(x, b6.is_valid()))
                 )

    for _, crossing in crossings:
        our_crossings.append(crossing)
        
        roads = w( b6.find(b6.within_cap( b6.find_feature(crossing), 1.0))
                  .filter(lambda v: b6.matches(v, b6.keyed("maxspeed")))
                  .map(lambda v: v.get_string("maxspeed"))
                 )

        if len(roads) == 1:
            # New feature: 'get_centroid'
            distance = w( b6.distance_meters( b6.find_feature(school).get_centroid()
                                            , b6.find_feature(crossing).get_centroid()
                                            ))

            # Tag the crossing with the distance to the school.
            # tag = b6.add_tag(school, b6.tag("distance-to-school", f"{int(distance)}"))
            # new_tags.append(tag)

            # Note: Would be ideal to not have to do this query again.
            kind = w( b6.find_feature(crossing) ).get_string("crossing_ref")

            results.append(
                { "Name": name
                , "school_feature_id": school
                , "crossing_feature_id": crossing
                , "Speed": roads[0][1]
                , "Distance": distance
                , "Kind": kind
                })

df = pd.DataFrame(results)
df

  0%|          | 0/161 [00:00<?, ?it/s]

,Name,school_feature_id,crossing_feature_id,Speed,Distance,Kind
0,Pirniehall Primary School,/point/openstreetmap.org/node/36150503,/point/openstreetmap.org/node/5803035196,20 mph,264.043296,
1,Pirniehall Primary School,/point/openstreetmap.org/node/36150503,/point/openstreetmap.org/node/3797106790,20 mph,143.883103,pelican
2,Pirniehall Primary School,/point/openstreetmap.org/node/36150503,/point/openstreetmap.org/node/3797106791,20 mph,132.092958,pelican
3,Pirniehall Primary School,/point/openstreetmap.org/node/36150503,/point/openstreetmap.org/node/746873944,20 mph,260.354669,zebra
4,Pirniehall Primary School,/point/openstreetmap.org/node/36150503,/point/openstreetmap.org/node/746873942,20 mph,250.149500,zebra
...,...,...,...,...,...,...
825,Basil Paterson School,/area/openstreetmap.org/way/1281993766,/point/openstreetmap.org/node/10305028448,20 mph,270.415540,
826,Basil Paterson School,/area/openstreetmap.org/way/1281993766,/point/openstreetmap.org/node/2342227330,20 mph,161.183965,zebra
827,Basil Paterson School,/area/openstreetmap.org/way/1281993766,/point/openstreetmap.org/node/10720095619,20 mph,177.124021,
828,Basil Paterson School,/area/openstreetmap.org/way/1281993766,/point/openstreetmap.org/node/9311191648,20 mph,75.941876,


In [44]:
min_df = df.loc[df.groupby('Name').Distance.idxmin()]
# min_df

Create tags for `min_df` by school

In [45]:
len( min_df.Distance )

124

In [257]:
new_tags = []

for _, row in min_df.iterrows():
    school_tag   = b6.add_tag(row.school_feature_id, b6.tag("#distance-to-crossing", f"{int(row.Distance)}"))
    crossing_tag = b6.add_tag(row.crossing_feature_id, b6.tag("#maxspeed", row.Speed))
    
    new_tags.append(school_tag)
    new_tags.append(crossing_tag)

In [331]:
# Write tags to a file.

# Note: Not used for now; prefer to do it dynamically.

# changes = b6.merge_changes(new_tags)
# w( b6.with_change(changes, lambda: b6.changes_to_file("add-school-distance-tags.yaml")) );

In [352]:
# min_df

In [353]:
# min_df[ min_df.Name.str.contains("Montessori") ]

In [340]:
min_df[ (min_df["Speed"] == "30 mph") ].sort_values(["Distance"])

,Name,school_feature_id,crossing_feature_id,Speed,Distance,Kind
103,Howdenhall & St Katharine's Special School,/point/openstreetmap.org/node/4046275141,/point/openstreetmap.org/node/4009560367,30 mph,45.773621,
356,Prestonfield Primary School,/area/openstreetmap.org/way/28791842,/point/openstreetmap.org/node/20935497,30 mph,69.906685,
708,Kaimes School,/area/openstreetmap.org/way/238481489,/point/openstreetmap.org/node/5781415836,30 mph,77.972063,puffin
370,Portobello High School,/area/openstreetmap.org/way/29260050,/point/openstreetmap.org/node/7894716013,30 mph,81.616804,
632,Oaklands School,/area/openstreetmap.org/way/124371000,/point/openstreetmap.org/node/8232347627,30 mph,83.693571,
479,Dunedin School,/area/openstreetmap.org/way/61319556,/point/openstreetmap.org/node/1765211426,30 mph,86.091387,
32,St Andrew's First Aid,/point/openstreetmap.org/node/3240991495,/point/openstreetmap.org/node/2265893165,30 mph,100.266567,
96,Broomhouse Primary School,/point/openstreetmap.org/node/4026681923,/point/openstreetmap.org/node/3254973256,30 mph,103.224214,puffin
99,St Joseph's RC Primary School,/point/openstreetmap.org/node/4026681924,/point/openstreetmap.org/node/3254973256,30 mph,112.753011,puffin
325,Gracemount High School,/area/openstreetmap.org/way/26654202,/point/openstreetmap.org/node/5781415836,30 mph,113.758203,puffin


In [355]:
# min_df[ (min_df["Speed"] == "20 mph") ].sort_values(["Distance"])

In [356]:
# min_df[ min_df.Name.str.contains("Portobello High School") ]

## Build b6 ui

In [349]:
name = "diagonal.works/edinburgh/school-crossing-analyis"

root_id               = b6.FeatureID(b6.FEATURE_TYPE_COLLECTION, name, 0)
distance_histogram_id = b6.FeatureID(b6.FEATURE_TYPE_COLLECTION, name, 1)
speed_histogram_id    = b6.FeatureID(b6.FEATURE_TYPE_COLLECTION, name, 2)
kind_histogram_id     = b6.FeatureID(b6.FEATURE_TYPE_COLLECTION, name, 3)

detail_root_id        = b6.FeatureID(b6.FEATURE_TYPE_COLLECTION, name, 4)
schools_collection_id = b6.FeatureID(b6.FEATURE_TYPE_COLLECTION, name, 5)

tabs = []

### Add the tags to the base world _at_ the point where we want to have our tabs

In [350]:
changes = b6.merge_changes(new_tags)
w(b6.add_world_with_change(root_id, changes));

### Histogram tab

In [351]:
# Distances
tagged_schools = b6.find(b6.keyed("#distance-to-crossing"))

distance_histogram = b6.histogram_with_id( tagged_schools.map(lambda v: v.get_float("#distance-to-crossing"))
                                         , distance_histogram_id
                                         )
distance_histogram_labels = b6.add_tag(distance_histogram_id, b6.tag("b6:label", "Distance to closest crossing"))


# Speeds on the street of the crossings
tagged_crossings = b6.find(b6.keyed("#maxspeed"))

speed_histogram = b6.histogram_with_id( tagged_crossings.map(lambda v: v.get_string("#maxspeed"))
                                      , speed_histogram_id
                                      )
speed_histogram_labels = b6.add_tag(speed_histogram_id, b6.tag("b6:label", "Max speed"))


# Kinds of the crossings
kind_histogram = b6.histogram_with_id( tagged_crossings.map(lambda v: v.get_string("crossing_ref"))
                                      , kind_histogram_id
                                      )
kind_histogram_labels = b6.add_tag(kind_histogram_id, b6.tag("b6:label", "Crossing kind"))

add_root = b6.add_collection(root_id, {}, b6.collection(
    b6.pair("centroid", emas_feature_id),
    b6.pair("docked", distance_histogram_id),
    b6.pair("docked", speed_histogram_id),
    b6.pair("docked", kind_histogram_id),
))

tabs.append( f"{root_id}:l:0:Histogram" )

all_changes = \
    [ add_root
    , distance_histogram
    , distance_histogram_labels
    , speed_histogram
    , speed_histogram_labels
    , kind_histogram
    , kind_histogram_labels
    ] 

w_root = b6.connect_insecure("0.0.0.0:8002", root=root_id)
changes = b6.merge_changes(all_changes)
w_root(b6.add_world_with_change(root_id, changes));

### Detail tab

In [352]:
# School-to-crossing
c = b6.collection(*[ b6.pair( b6.find_feature(r.school_feature_id)
                            , b6.find_feature(r.crossing_feature_id)
                            ) for _, r in min_df.iterrows()])
add_collection = b6.add_collection(schools_collection_id, {"b6-title": b6.tag("b6:title", "Schools and closest crossing")}, c)


add_detail_root = b6.add_collection(detail_root_id, {}, b6.collection(
    b6.pair("centroid", emas_feature_id),
    b6.pair("docked", schools_collection_id),
))

tabs.append(f"{detail_root_id}:l:1:Details")

w_detail = b6.connect_insecure("0.0.0.0:8002", root=detail_root_id)
changes = b6.merge_changes([ add_detail_root, add_collection ])
w_detail(b6.add_world_with_change(detail_root_id, changes));

In [353]:
# tabs

In [354]:
# tabs[0].split(":")
# id:side:index:name

### Show the map URL

In [355]:
tab_expression = ",".join(tabs)
print(f"http://localhost:8001/?r={root_id}&t={tab_expression}")

http://localhost:8001/?r=/collection/diagonal.works/edinburgh/school-crossing-analyis/0&t=/collection/diagonal.works/edinburgh/school-crossing-analyis/0:l:0:Histogram,/collection/diagonal.works/edinburgh/school-crossing-analyis/4:l:1:Details


## Todo

- [x] Add world in b6 and center around EMAS
- [ ] Add dock to click on school and see closest crossing
- [ ] Colour the schools
- [ ] (Bonus) Emoji's for the crossing
- [x] Histogram for the breakdowns
- [ ] (Bonus) Tabs for "top 10 best" and "top 10 worst"
- [x] Histogram for the crossing kinds
- [x] Histogram for the speeds
- [ ] Comparative analysis to Glasgow
- [ ] (Bonus) Is there data on if there is a "Lollipop person" present at the crossings?

### Wildcard ideas

- [ ] Draw a large circle around the schools just to see them.